#### Challenge

In [ ]:
from utils import listener
from sage.all import *

FLAG = b"crypto{????????????????????????}"

# dimension
n = 64
# plaintext modulus
p = 257
# ciphertext modulus
q = 0x10001

V = VectorSpace(GF(q), n)
S = V.random_element()

def encrypt(m):
    A = V.random_element()
    b = A * S + m
    return A, b


class Challenge:
    def __init__(self):
        self.before_input = "Would you like to encrypt your own message, or see an encryption of a character in the flag?\n"

    def challenge(self, your_input):
        if 'option' not in your_input:
            return {'error': 'You must specify an option'}

        if your_input['option'] == 'get_flag':
            if "index" not in your_input:
                return {"error": "You must provide an index"}
                self.exit = True

            index = int(your_input["index"])
            if index < 0 or index >= len(FLAG) :
                return {"error": f"index must be between 0 and {len(FLAG) - 1}"}
                self.exit = True

            A, b = encrypt(FLAG[index])
            return {"A": str(list(A)), "b": str(int(b))}

        elif your_input['option'] == 'encrypt':
            if "message" not in your_input:
                return {"error": "You must provide a message"}
                self.exit = True

            message = int(your_input["message"])
            if message < 0 or message >= p:
                return {"error": f"message must be between 0 and {p - 1}"}
                self.exit = True

            A, b = encrypt(message)
            return {"A": str(list(A)), "b": str(int(b))}

        return {'error': 'Unknown action'}


import builtins; builtins.Challenge = Challenge # hack to enable challenge to be run locally, see https://cryptohack.org/faq/#listener
listener.start_server(port=13411)


In [9]:
from pwn import *
from sage.all import *
from Crypto.Util.number import *

host = 'socket.cryptohack.org'
port = 13411
r = remote(host, port, level='DEBUG')
r.recvuntil(b'Would you like to encrypt your own message, or see an encryption of a character in the flag?\n')

import json

n = 64
p = 257
q = 0x10001

## recover S
rows = []
rhs = []
for i in range(n):
    payload = {
        "option": "encrypt",
        "message": "0"
    }

    payload = json.dumps(payload).encode()
    r.sendline(payload)

    response = json.loads(r.recvline().strip().decode())
    A = eval(response["A"])
    b = int(response["b"])

    rows.append(A)
    rhs.append(b)

A = Matrix(GF(q), rows)
b = vector(GF(q), rhs)

S = A ** (-1) * b
print(S)

flag = ""
i = 0

while True:
    payload = {
        "option": "get_flag",
        "index": str(i)
    }

    payload = json.dumps(payload).encode()
    r.sendline(payload)

    response = json.loads(r.recvline().strip().decode())
    b = int(response["b"])
    A = eval(response["A"])
    A = vector(GF(q), A)
    
    x = b - (A * S)
    print(x)

    flag += chr(x)
    print(flag)

    i += 1




[x] Opening connection to socket.cryptohack.org on port 13411
[x] Opening connection to socket.cryptohack.org on port 13411: Trying 134.122.111.232
[+] Opening connection to socket.cryptohack.org on port 13411: Done
[DEBUG] Received 0x5d bytes:
    b'Would you like to encrypt your own message, or see an encryption of a character in the flag?\n'
[DEBUG] Sent 0x26 bytes:
    b'{"option": "encrypt", "message": "0"}\n'
[DEBUG] Received 0x1ce bytes:
    b'{"A": "[23466, 25591, 25052, 17576, 51406, 8270, 16748, 55109, 60584, 26586, 11823, 11538, 7170, 21624, 4886, 60360, 54808, 41843, 45258, 30500, 22240, 64115, 36967, 41224, 33328, 21250, 2397, 9501, 21968, 54657, 11381, 35249, 46967, 47852, 38760, 15692, 56094, 17410, 39261, 29853, 3390, 51475, 7098, 16930, 32453, 56455, 29546, 40675, 20632, 28607, 23736, 23450, 37716, 50382, 57046, 8560, 10252, 50816, 55574, 49728, 46475, 9878, 9286, 35046]", "b": "17364"}\n'
[DEBUG] Sent 0x26 bytes:
    b'{"option": "encrypt", "message": "0"}\n'
[DEBUG] 

KeyError: 'b'

In [ ]:
## crypto{linear_algebra_is_useful}